In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pillow_heif
from PIL.ExifTags import TAGS
import piexif
from pillow_heif import register_heif_opener

os.chdir("/home/michael/AmbientLight/")

from src.utils import extract_specific_metadata, create_hdr_from_images, estimate_overall_lux

# create_hdr_from_images(image_paths)



In [2]:

image_paths = [
    "test_photos_heic/face-2.HEIC",
    "test_photos_heic/face0.HEIC",
    "test_photos_heic/face+2.HEIC"
]

image_paths = [
    "test_photos_heic/table-2.HEIC",
    "test_photos_heic/table0.HEIC",
    "test_photos_heic/table+2.HEIC"
]

image_paths = [
    "disbursed_heic/IMG_3786.HEIC",
    "disbursed_heic/IMG_3787.HEIC",
    "disbursed_heic/IMG_3788.HEIC",
    "disbursed_heic/IMG_3789.HEIC",
    "disbursed_heic/IMG_3790.HEIC",
    "disbursed_heic/IMG_3791.HEIC",
    "disbursed_heic/IMG_3792.HEIC",
    
]

metadata_list = extract_specific_metadata(image_paths)
overall_lux_estimates = estimate_overall_lux(metadata_list)


In [3]:
metadata_list

[{'filename': 'disbursed_heic/IMG_3786.HEIC',
  'ExposureTime': 0.016666666666666666,
  'FNumber': 2.2,
  'ISOSpeedRatings': 400,
  'BrightnessValue': 1.7691698038438677,
  'ExposureBiasValue': 0.0,
  'calculatedExposure': 1.3774104683195592,
  'lux_unadjusted': 181.50000000000003,
  'lux_adjusted': 134.8675464088753,
  'mean_pixel_value': 111.8375432756212},
 {'filename': 'disbursed_heic/IMG_3787.HEIC',
  'ExposureTime': 0.016666666666666666,
  'FNumber': 1.7799999713880652,
  'ISOSpeedRatings': 320,
  'BrightnessValue': 1.2530090165653176,
  'ExposureBiasValue': 0.0,
  'calculatedExposure': 1.6832892011080116,
  'lux_unadjusted': 148.51874522538344,
  'lux_adjusted': 109.9585384317252,
  'mean_pixel_value': 111.65238624148898},
 {'filename': 'disbursed_heic/IMG_3788.HEIC',
  'ExposureTime': 0.016666666666666666,
  'FNumber': 1.7799999713880652,
  'ISOSpeedRatings': 800,
  'BrightnessValue': 0.3303541404881157,
  'ExposureBiasValue': 0.0,
  'calculatedExposure': 4.20822300277003,
  'l

In [4]:
overall_lux_estimates
overall_lux_estimates['weighted_mean_adjusted']

#use adjusted
# median is stupid
# adjusted mean
# use weighting

# weighted_mean_adjusted 


{'median_adjusted': 48.009220123291016,
 'mean_adjusted': 69.81218719482422,
 'weighted_median_adjusted': 48.00922009066983,
 'weighted_mean_adjusted': 71.0287857055664,
 'median_unadjusted': 95.05199432373047,
 'mean_unadjusted': 103.15831756591797,
 'weighted_median_unadjusted': 95.05199694424539,
 'weighted_mean_unadjusted': 102.8966064453125}